## Installing Pytorch

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x5919e000 @  0x7f427a7b62a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


## Building Fairseq

In [2]:
!git clone https://github.com/pytorch/fairseq.git

Cloning into 'fairseq'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 3094 (delta 13), reused 8 (delta 7), pack-reused 3063
Receiving objects: 100% (3094/3094), 3.14 MiB | 3.32 MiB/s, done.
Resolving deltas: 100% (2247/2247), done.


In [0]:
import os
os.chdir("fairseq/")

In [4]:
!pip install -r requirements.txt
%run -i 'setup.py' build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.6
creating build/lib.linux-x86_64-3.6/tests
copying tests/test_iterators.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_noising.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_convtbc.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_dictionary.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_character_token_embedder.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_average_checkpoints.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_sequence_scorer.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_label_smoothing.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_utils.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_binaries.py -> build/lib.linux-x86_64-3.6/tests
copying tests/__init__.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_backtranslation_dataset.py -> build/lib.linux-x86_64-3.6/tests
copying tests/test_sequ

## Model for Afrikaans
### Data Preprop
Upload the data in the form train.en, train.tn, valid.en, valid.tn, test.en, test.tn.

### Subword

In [5]:
!git clone https://github.com/rsennrich/subword-nmt

Cloning into 'subword-nmt'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 485 (delta 0), reused 1 (delta 0), pack-reused 481
Receiving objects: 100% (485/485), 205.64 KiB | 388.00 KiB/s, done.
Resolving deltas: 100% (287/287), done.


In [0]:
os.chdir('../')

In [0]:
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

# Learn a vocabulary using 40,000 merge operations
!fairseq/subword-nmt/learn_bpe.py -s 4000 <combine.txt> enaf.codes

# Apply the vocabulary to the training file
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

In [0]:
os.chdir('fairseq/')

In [0]:
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

Namespace(alignfile=None, destdir='data-bin/enaf', joined_dictionary=False, nwordssrc=-1, nwordstgt=-1, only_source=False, output_format='binary', padding_factor=8, source_lang='en', srcdict=None, target_lang='af', testpref='../test', tgtdict=None, thresholdsrc=0, thresholdtgt=0, trainpref='../train', validpref='../valid', workers=1)
| [en] Dictionary: 3511 types
| [en] ../train.en: 37219 sents, 790613 tokens, 0.0% replaced by <unk>
| [en] Dictionary: 3511 types
| [en] ../valid.en: 12953 sents, 383339 tokens, 0.114% replaced by <unk>
| [en] Dictionary: 3511 types
| [en] ../test.en: 3000 sents, 76516 tokens, 0.0562% replaced by <unk>
| [af] Dictionary: 3967 types
| [af] ../train.af: 37219 sents, 813055 tokens, 0.0% replaced by <unk>
| [af] Dictionary: 3967 types
| [af] ../valid.af: 12953 sents, 403141 tokens, 0.344% replaced by <unk>
| [af] Dictionary: 3967 types
| [af] ../test.af: 3000 sents, 78473 tokens, 0.0624% replaced by <unk>
| Wrote preprocessed data to data-bin/enaf


### Train Model

In [0]:
!mkdir -p chckpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir ckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

### Testing
Generate translations from the test data. Calculate the Bleu score.

In [0]:
# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**8k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 8000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checkpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path checkpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**12k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 12000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir ceckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path ceckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**16k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 16000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir chekpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path chekpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**20k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 20000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir chckpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path chckpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**24k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 24000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path checpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**28k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 28000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checpin/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path checpin/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**32k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 32000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path checpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**36k**

In [0]:
#os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 36000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir checpoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path checpoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

**40k**

In [0]:
os.chdir('../')
!cat enaf_parallel.train.en enaf_parallel.train.af > combine.txt

!fairseq/subword-nmt/learn_bpe.py -s 40000 <combine.txt> enaf.codes
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.en > train.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.train.af > train.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.en > valid.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.dev.af > valid.af
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.en > test.en
!fairseq/subword-nmt/apply_bpe.py -c enaf.codes < enaf_parallel.test.af > test.af

os.chdir('fairseq/')
TEXT=".."
%run 'preprocess.py' --source-lang en --target-lang af --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test --destdir data-bin/enaf

!mkdir -p checkpoint/fconv
!python train.py data-bin/enaf \
    --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
    --arch fconv --save-dir chepoint/fconv

# for sentencepiece remove ▁
output = %run 'generate.py' data-bin/enaf --path chepoint/fconv/checkpoint_best.pt --beam 5 --batch-size 128 --remove-bpe --source-lang en --target-lang af

## Results

*   40k: 19.00
*   36k: 16.28
*   32k: 20.61
*   28k: 20.91
*   24k: 22.80
*   20k: 13.46
*   16k: 23.06
*   12k: 20.88
*     8k: 23.20
*     4k: 25.04

